In [ ]:
import os
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline
# plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['font.size'] = 7
plt.rcParams['font.sans-serif'] = [u'Arial']



LINEWIDTH = 0.25

plt.rcParams['lines.linewidth'] = LINEWIDTH
plt.rcParams['axes.linewidth'] = LINEWIDTH
plt.rcParams['xtick.major.width'] = LINEWIDTH
plt.rcParams['ytick.major.width'] = LINEWIDTH

plt.rcParams['legend.fancybox'] = False
plt.rcParams['legend.frameon'] = False


import brewer2mpl
bmap = brewer2mpl.get_map('Set1', 'qualitative', 5)
colors = bmap.mpl_colors

plt.rcParams['axes.color_cycle'] = colors



from scipy.stats import ks_2samp
from sklearn.neighbors import KernelDensity

import pandas as pd


from context import src, utils
from src.analyzer import DataAnalyzer, plot_fill_between
from utils.plot_utils import label_subplot

# output_dir = '../output/motion_benefit6'
# output_dir = '../output/motion_benefit_nom'

In [ ]:
# d = plt.rcParams
# for k in d.keys():
#     if k.startswith('font'):
#         print k, d[k]

# plt.rcParams.keys()

In [ ]:
data_fns = []
# for output_dir in ['../output/motion_benefit_nom', '../output/motion_benefit6']:
# for output_dir in ['../output/motion_benefit_drop']:
for output_dir in ['../output/motion_benefit_drop_best_dc']:
    data_fns.extend(
        [os.path.join(output_dir, fn) 
               for fn in os.listdir(output_dir) 
               if fn.endswith('.h5')]
        )
data_fns.sort()
len(data_fns)

In [ ]:
da_ = map(DataAnalyzer.fromfilename, data_fns)

In [ ]:
for da in da_:
    da.s_range = 'pos'

In [ ]:
out_ = []
for da in da_:
    out = {}
    out['inference_mode'] = da.data['EM_data/mode']
    out['gen_mode'] = da.data['motion_gen']['mode']   # True eye movements or no eye movements
    try:
        out['gen_dc'] = da.data['motion_gen']['dc']
    except KeyError:
        out['gen_dc'] = 0.0
    try:
        out['drop_prob'] = da.data['drop_prob']
    except KeyError:
        out['drop_prob'] = 0.0
    out['prior_dc'] = da.data['motion_prior']['dc']
    out['ds'] = da.data['ds'] # Image size
    for t, snr_t in zip(da.time_list(), da.snr_list()):
        out[t] = snr_t

    out_.append(out)

In [ ]:
t = da.time_list()
data = pd.DataFrame(out_)

grouping_columns = ['inference_mode', 'gen_mode', 'prior_dc', 'ds', 'drop_prob']
grouped = pd.groupby(data, grouping_columns)

EXP = 'Experiment'
NOM = 'Diffusion'  # no motion

# label_ = ['Motion', 'No-motion']

In [ ]:
def nice_print(lst):
    l1 = []
    for l in lst:
        if isinstance(l, str):
            l1.append(l)
        elif isinstance(l, float):
            l1.append('{:6.2f}'.format(l))
        elif isinstance(l, int):
            l1.append('{:02}'.format(l))
        else:
            l1.append(str(l))
    return ' | '.join(l1)

In [ ]:
print ['Group'] + grouping_columns + ['SNR']
for i, (name, group) in enumerate(grouped):
    mode, gen_mode, prior_dc, ds, drop_prob = name
#     if ds != 0.4:
#         continue
    print nice_print((i,) + name + (group[list(t)[-1]].mean(),))

In [ ]:
df1 = grouped.aggregate(np.mean)[['gen_dc', t[-1]]]
df1.reset_index(inplace=True)

In [ ]:
ax = plt.axes()
for name, group in df1.groupby('drop_prob'):
    group.plot(x='gen_dc', y=t[-1], label='Drop Prob: {:.2f}'.format(name), ax=ax)
ax.set_ylabel('SNR')
plt.savefig('../output/drop_prob.pdf')

In [ ]:
# Plot mean SNR as a function of DC for the two conditions

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
ax.set_title('SNR as a function of time')
for name, group in grouped:
    inf_mode, gen_mode, prior_dc, ds = name
#     label = 'Inf: {} Gen: {} GenDC:  InfDC: {} ds: {:.2f}'.format(*name)
    label = {EXP: 'Motion', NOM: 'No Motion'}[gen_mode] + ' | ' + inf_mode
    plot_fill_between(ax, t, group[list(t)], label=label, k=0.5)

ax.set_xlabel('time (ms)')
ax.legend(loc='upper left')
# plt.savefig(os.path.join('../output', 'snr_time_no_motion_vs_motion_1.pdf'))

In [ ]:
final_snrs = [group[list(t)[-1]].values for k, group in grouped]

res = ks_2samp(final_snrs[0], final_snrs[1])
label_ = [k[1] for k, group in grouped]

res

In [ ]:
plt.figure(figsize=(4, 4))
X_plot = np.linspace(0, 20, 1000)[:, np.newaxis]
for final_snr, label in zip(final_snrs, label_):
    log_dens = KernelDensity().fit(final_snr[:, np.newaxis]).score_samples(X_plot)
    plt.plot(X_plot, np.exp(log_dens), c=None, label=label)
plt.xlabel('SNR')
plt.ylabel('Normalized Density')
plt.title('Density of SNR, p = {:.1E}'.format(res.pvalue))
plt.legend()
plt.tight_layout()
# plt.savefig(os.path.join(output_dir, 'motion_benefit_density.png'), dpi=200)

# plt.hist(final_snrs, bins=20)

In [ ]:
idx = [data[
        (data['gen_mode'] == key) & 
        (data['ds'] == 0.4) & 
        (data['inference_mode'] == 'EM')
    ].index.values for key in [EXP, NOM]]
exp_idx, nom_idx = idx

# np.random.shuffle(exp_idx)
# np.random.shuffle(nom_idx)

In [ ]:
da = da_[exp_idx[2]]

In [ ]:
da.plot_em_estimate(-1)

In [ ]:
def plot_results(ax, grouped):
    from matplotlib.patches import Rectangle
    c_ = ['b', 'g']
#     hatch_ = ['o', 'x']
    hatch_ = [None, None]
    alpha = 0.5
    label_ = []
    rect_ = []
    for name, group in grouped:
        inf_mode, gen_mode, prior_dc, ds = name
        label = ({EXP: 'M', NOM: 'NoM'}[gen_mode] + ' | ' + 
                 {'EM': 'EM ', 'NoMotion': 'NoM'}[inf_mode])
        plot_fill_between(ax, t, group[list(t)], label=label, c=None, 
                          k=0.5)
        c = ax.get_lines()[-1].get_color()
        rect_.append(Rectangle((0, 0), 1, 1, fc=c, hatch=None))
        label_.append(label)
    ax.legend(rect_, label_, loc='upper left', prop={'size': 7}, labelspacing=0.2)


In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(3.5, 3.5))

da_exp, da_nom = [da_[idx_[0]] for idx_ in [exp_idx, nom_idx]]

ax = axes[0, 0]
da_exp.plot_image_and_rfs(fig, ax, -1)
a = 4; ax.set_xlim([-a, a]); ax.set_ylim([-a, a])
ax.xaxis.set_major_locator(MaxNLocator(3, symmetric=True))
ax.yaxis.set_major_locator(MaxNLocator(3, symmetric=True))



ax = axes[0, 1]
plot_results(ax, grouped)
ax.set_xlim([0., 700.])
ax.set_xticks([0, 300, 600])
ax.set_title('SNR vs time (ms)')
# ax.set_xlabel('time (ms)')
# ax.set_ylabel('SNR')

# ax.xaxis.set_major_locator(MaxNLocator(3))
ax.yaxis.set_major_locator(MaxNLocator(4))

for v, da, label in zip(
    [0, 1],
    [da_exp, da_nom],
    ['Motion', 'No Motion']):
    ax = axes[1, v]
    da.plot_image_estimate(fig, ax, da.N_itr - 1,
                           colorbar=False)
    ax.set_title('{}: SNR = {:.2f}'.format(label, da.snr_one_iteration(da.N_itr - 1)))

    ax.xaxis.set_major_locator(MaxNLocator(3))
    ax.yaxis.set_major_locator(MaxNLocator(3))


for i, ax in enumerate(axes.flat):
    ax.set_title(ax.get_title(), fontdict={'size': 7})
    label=chr(ord('a') + i)
    label_subplot(fig=fig, ax=ax, label=label, dx=0.08, dy=0.03)

    
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_visible(False)
    
plt.tight_layout(pad=0.5)
plt.savefig(os.path.join('../output', 'motion_benefit_drop.pdf'), dpi=200)

In [ ]:
# plt.figure(figsize=(4, 4))
# plt.title('SNR as a function of time')
# alpha = 0.5
# for c, (name, group) in zip(c_, grouped):
#     inf_mode, gen_mode, prior_dc, ds = name
#     label = {EXP: 'Motion', NOM: 'No Motion'}[gen_mode]
#     plt.plot(t, group[list(t)].T.iloc[:, 0], c=c, label=label, alpha=alpha)
#     plt.plot(t, group[list(t)].T.iloc[:, 1:], c=c, alpha=alpha);
#     plt.xlabel('time (ms)')
# plt.legend(loc='upper left')
# # plt.savefig(os.path.join(output_dir, 'snr_time_no_motion_vs_motion_0.png'), dpi=200)

In [ ]:
NOM

In [ ]:
idx = data[
        (data['gen_mode'] == NOM) & 
#         (data['ds'] == 0.4) & 
        (data['inference_mode'] == 'NoMotion')
    ].index.values
print idx

In [ ]:
da = da_[idx[0]]

In [ ]:
fig, ax = plt.subplots(1, 1)

da.plot_image_estimate(fig=fig, ax=ax, q=-1)

In [ ]:
da.plot_em_estimate(-1)
# plt.savefig(os.path.join(output_dir, 'ds04_e_reconstruction.png'), dpi=200)

In [ ]:
fig, ax = plt.subplots(figsize=(2, 2))
ax.set_xlabel(None, fontsize=7)
ax.set_ylabel(None, fontsize=7)
da.plot_image_and_rfs(fig, ax)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(7)
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(7)
ax.title.set_fontsize(7)



plt.title('E Projected onto the Retina \n E thickness: {:.2f} arcmin \n Neuron Spacing {:.2f} arcmin'.format(
        2 * da.data['ds'], da.data['de']), fontsize=7)
a = 6; plt.xlim([-a, a]); plt.ylim([-a, a])


plt.tight_layout(pad=0.1)

# plt.savefig(os.path.join(output_dir, 'motion_benefit_e.pdf'))

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(4 * 2, 3.5 * 5))
for i, idx in enumerate(nom_idx[0:10]):
#     plt.subplot(5, 2, i + 1)
    da_[idx].plot_image_estimate(fig, axes.flat[i], -1)
plt.savefig(os.path.join(output_dir, 'e_reconstructions_no_motion.png'), dpi=200)

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(4 * 2, 3.5 * 5))
for i, idx in enumerate(exp_idx[0:10]):
    da_[idx].plot_image_estimate(fig, axes.flat[i], -1)
plt.savefig(os.path.join(output_dir, 'e_reconstructions_motion.png'), dpi=200)

In [ ]:
plt.figure(figsize=(20, 20))
for i, t in enumerate(range(0, 140, 10)):
    plt.subplot(5, 5, i + 1)
    da.plot_image_estimate(t)

# Group SNR plots by image size and diffusion constant

In [ ]:
out_ = []
for pkl_fn in pkl_fns:
    da = DataAnalyzer.fromfilename(pkl_fn)
    out = [da.data['ds'], da.data['motion_gen']['dc']]
    out = out + da.SNR_list()
    out_.append(out)

In [ ]:
t = da_[0].time_list()

In [ ]:
data = pd.DataFrame.from_records(out_, columns=['ds', 'dc_gen'] + list(t))
# data = data[data['ds'] == 0.5]

In [ ]:
grouped = pd.groupby(data, ['ds', 'dc_gen'])
c_ = ['g', 'b', 'r', 'y', 'm', 'c']

In [ ]:
plt.figure(figsize=(10, 7))
plt.title('SNR as a function of time')
for c, (name, group) in zip(c_, grouped):
    label = 'ds={}, dc={}'.format(*name)
    plot_fill_between(t, group[list(t)], label=label, c=c)
plt.legend(loc='upper left')
# plt.savefig(os.path.join(output_dir, 'motion_benefit.png'), dpi=200)